In [136]:
import pandas as pd
import numpy as np
import folium
from branca.element import Figure
import json
import openrouteservice as ors
import time

In [137]:
ors_key = "5b3ce3597851110001cf624890d3df4ec58741129d23e67c5af0e306"
client = ors.Client(key=ors_key)

In [138]:
routes = {}
for i in range(1, 6):
    arcs = pd.read_csv(f'/home/meet/Documents/Internship/Solid-Waste-Collection/Data/Dynamic Data/Multiple Trucks/Best case/Visited Truck 1/visited_truck1_{i}_0.9_0.1.csv')
    routes[i - 1] = arcs.Node.astype('int').tolist()

In [139]:
routes

{0: [0,
  268,
  119,
  90,
  123,
  32,
  269,
  75,
  162,
  131,
  9,
  283,
  109,
  21,
  295,
  137,
  1,
  59,
  40,
  0],
 1: [0,
  228,
  85,
  104,
  240,
  132,
  190,
  129,
  290,
  26,
  152,
  87,
  220,
  285,
  3,
  145,
  250,
  0],
 2: [0,
  249,
  16,
  113,
  156,
  55,
  167,
  221,
  149,
  43,
  206,
  262,
  81,
  239,
  255,
  74,
  72,
  103,
  244,
  175,
  251,
  0],
 3: [0,
  203,
  199,
  218,
  107,
  271,
  60,
  44,
  2,
  232,
  230,
  56,
  224,
  138,
  231,
  217,
  88,
  273,
  0],
 4: [0,
  159,
  11,
  93,
  48,
  282,
  19,
  211,
  10,
  22,
  133,
  261,
  98,
  29,
  35,
  49,
  27,
  17,
  276,
  139,
  116,
  46,
  23,
  0]}

In [140]:
df = pd.read_csv('/home/meet/Documents/Internship/Solid-Waste-Collection/Data/Bin Locations.csv', index_col='id')
latlong_coord_dict = {}
longlat_coord_dict = {}
for key,value in routes.items():
    latlong_path = []
    longlat_path = []
    for step in value:
        latlong_path.append([df.loc[step, 'y'],df.loc[step, 'x']])
        longlat_path.append([df.loc[step, 'x'],df.loc[step, 'y']])
    
    latlong_coord_dict[key] = latlong_path
    longlat_coord_dict[key] = longlat_path


In [141]:
routes_dict_ward1 = {}
total_distances_dict = {}

for key, value in latlong_coord_dict.items():
    time.sleep(0.5)
    direction_api = client.directions(
        coordinates = longlat_coord_dict[key],
        profile = 'driving-car',
        format = 'geojson'
    )
    coords = direction_api['features'][0]['geometry']['coordinates']
    routes_dict[key] = [[coord[1], coord[0]] for coord in coords]
    total_distances_dict[key] = direction_api['features'][0]['properties']['summary']['distance']



In [142]:
total_distances_dict

{0: 48988.2, 1: 28094.0, 2: 73979.3, 3: 51331.7, 4: 97060.6}

In [143]:
chandigarh_map = folium.Map(location=[30.741482, 76.768066], zoom_start = 11.3)

In [144]:
# Creating feature groups

# Ward 1
cargo1 = folium.FeatureGroup("Cargo 1")
cargo2 = folium.FeatureGroup("Cargo 2")
cargo3 = folium.FeatureGroup("Cargo 3")
cargo4 = folium.FeatureGroup("Cargo 4")
cargo5 = folium.FeatureGroup("Cargo 5")

# Ward 2
cargo6 = folium.FeatureGroup("Cargo 6")
cargo7 = folium.FeatureGroup("Cargo 7")
cargo8 = folium.FeatureGroup("Cargo 8")
cargo9 = folium.FeatureGroup("Cargo 9")
cargo10 = folium.FeatureGroup("Cargo 10")

# Ward 2
cargo11 = folium.FeatureGroup("Cargo 11")
cargo12 = folium.FeatureGroup("Cargo 12")
cargo13 = folium.FeatureGroup("Cargo 13")
cargo14 = folium.FeatureGroup("Cargo 14")
cargo15 = folium.FeatureGroup("Cargo 15")

# Create Marker
folium.CircleMarker(
    location=latlong_coord_dict[0][0],
    popup = 'Depot',
    tooltip = '<strong>Click here to see Popup</strong>',
    color = 'black',
    radius = 1,
    fill = True, 
    fill_color = color
    ).add_to(chandigarh_map)

In [145]:
points = {}
for key in np.unique(df.Ward):
    if key == -1:
        continue
    y = df[df.Ward == key].loc[:, 'y'].tolist()
    x = df[df.Ward == key].loc[:, 'x'].tolist()
    y_x = []
    for i in range(len(y)):
        y_x.append([y[i], x[i]])
    points[key] = y_x

In [146]:
marker_colors = ['red', 'green', 'darkblue']

#add others markers
for key, value in points.items():
    color = marker_colors[key]
    print(color, key)

    for step in range(len(value)):
        # popup = 'Cargo {0} - Destination {1}'.format(key + 1, step)
        folium.CircleMarker(
            radius = 1,
            location = points[key][step],
            color = color,
            fill = True,
            fill_color = color
            ).add_to(chandigarh_map)

red 0
green 1
darkblue 2


In [147]:
routes_dict.keys()

dict_keys([0, 1, 2, 3, 4])

In [148]:
# Adding lines to the different feautre groups
route_line_1 = folium.vector_layers.PolyLine(
    routes_dict[0],
    popup = '<b>Path of Truck 1</b>',
    tooltip = 'Truck 1',
    color = 'red',
    weight = 4
).add_to(cargo1)
route_line_2 = folium.vector_layers.PolyLine(
    routes_dict[1],
    popup = '<b>Path of Truck 2</b>',
    tooltip = 'Truck 2',
    color = 'green',
    weight = 4
).add_to(cargo2)
route_line_3 = folium.vector_layers.PolyLine(
    routes_dict[2],
    popup = '<b>Path of Truck 3</b>',
    tooltip = 'Truck 3',
    color = 'blue',
    weight = 4
).add_to(cargo3)
route_line_4 = folium.vector_layers.PolyLine(
    routes_dict[3],
    popup = '<b>Path of Truck 4</b>',
    tooltip = 'Truck 4',
    color = 'lightblue',
    weight = 4
).add_to(cargo4)
route_line_5 = folium.vector_layers.PolyLine(
    routes_dict[4],
    popup = '<b>Path of Truck 5</b>',
    tooltip = 'Truck 5',
    color = 'purple',
    weight = 4
).add_to(cargo5)

In [149]:
cargo1.add_to(chandigarh_map)
cargo2.add_to(chandigarh_map)
cargo3.add_to(chandigarh_map)
cargo4.add_to(chandigarh_map)
cargo5.add_to(chandigarh_map)

folium.LayerControl().add_to(chandigarh_map)

In [150]:
chandigarh_map